In [4]:
const the_config = {
  refresh_interval: '$fn ({vars, hass, visible_range}) => "auto"',
  offset:
    '$fn ({vars, hass, visible_range}) => {vars.a_var = "from offset"; return 5;}',
  hours_to_show: "$fn ({vars, hass, visible_range, xs, ys}) => 5",
  entities: [
    {
      entity: "sensor.temperature",
      attribute: '$fn ({vars, hass, visible_range}) => "temperature"',
      offset: 5,
      unit_of_measurement: "W",
      text: "$fn ({vars, hass, visible_range, xs, ys}) => { vars.ys = ys; return ys.map(y => 'y=${y}')}",
      filters: [
        {
          add: "$fn ({vars, hass, visible_range}) => 5",
        },
      ],
    },
  ],
  defaults: {
    entity: {
      internal: "$fn ({vars, hass, visible_range, xs, ys}) => ys.map(y => y%2)",
      show_value: true,
    },
    yaxes: {
      autorange: "$fn ({vars, hass, visible_range}) => true",
      fixedrange: false,
    },
  },
  layout: {
    title: "$fn ({vars, hass, visible_range}) => visible_range",
    legend: { traceorder: "normal" },
  },
  disable_pinch_to_zoom: "$fn ({vars, hass, visible_range}) => vars.a_var",
  stuff_coming_from_fetched_data: "$fn ({vars, hass, visible_range}) => vars",
};


In [5]:
const defaultEntity = {
  hovertemplate: `<b>%{customdata.name}</b><br><i>%{x}</i><br>%{y}%{customdata.unit_of_measurement}<extra></extra>`,
  mode: "lines",
  show_value: false,
  line: {
    width: 1,
    shape: "hv",
    // color: colorScheme[entityIdx % colorScheme.length],
  },
  internal: false,
  offset: "0s",
  // extend_to_present: entityIn.extend_to_present ?? !statisticConfig.statistic,
};
const defaultYaml = {
  title: "",
  hours_to_show: 1,
  refresh_interval: "auto",
  color_scheme: "category10",
  offset: "0s",
  no_theme: false,
  no_default_layout: false,
  significant_changes_only: false,
  minimal_response: true,
  disable_pinch_to_zoom: false,
  defaults: {
    entity: {},
    yaxes: {},
  },
  config: {
    displaylogo: false,
    scrollZoom: true,
    modeBarButtonsToRemove: ["resetScale2d", "toImage", "lasso2d", "select2d"],
  },
  layout: {},
};


In [7]:
import getThemedLayout from "./themed-layout";

import merge from "lodash/merge";
import get from "lodash/get";

async function fetchData(entityConfig, hours_to_show, globalOffset) {
  return {
    xs: [1, 2, 3, 4],
    ys: [21, 22, 23, 24],
    meta: {},
    // etc
  };
}

function isObjectOrArray(value) {
  return typeof value == "object" && !(value instanceof Date);
}
class ConfigParser {
  private partiallyParsedConfig: {};
  private busy = true;
  public get config() {
    if (this.busy) throw new Error("busy");
    return this.partiallyParsedConfig;
  }

  private getFromPartial(p: { path: string; callingPath: string }) {
    const value = get(this.partiallyParsedConfig, p.path);
    if (typeof value === "string" && value.startsWith("$fn")) {
      throw new Error(
        `Since [${p.path}] is a $fn, it has to be defined before [${p.callingPath}]`
      );
    }
    return value;
  }
  private async evalNode({
    parent,
    path,
    key,
    value,
    fnParam,
  }: {
    parent: object;
    path: string;
    key: string;
    value: any;
    fnParam: object;
  }) {
    if (path.match(/^defaults$/)) return;

    if (typeof value === "string" && value.startsWith("$fn")) {
      value = eval(value.slice(3));
    }
    if (typeof value === "function") value = value(fnParam);

    if (path.match(/^entities\.\d+$/)) {
      const keys = Object.keys(value);
      const requiredKeys = [
        "entity",
        "offset",
        "statistic",
        "period",
        "attribute",
      ];
      const fetchAt = Math.max(...requiredKeys.map((k) => keys.indexOf(k)));
      const me: any = (parent[key] = {});
      for (let i = 0; i < keys.length; i++) {
        const childKey = keys[i];
        const childValue = value[childKey];
        if (i === fetchAt) {
          const hours_to_show = this.getFromPartial({
            path: "hours_to_show",
            callingPath: path,
          });
          const globalOffset = this.getFromPartial({
            path: "offset",
            callingPath: path,
          });
          /**
           TODOs: 
            * parse durations for both offsets
            * parseColorScheme and use in entity
            * parseStatistics
            * filters
            * add logic from getLayout
            * getDataAndUnits
            * fetch
            * plot
            * separate and put in a file
           *  */ 
          const data = await fetchData(me, hours_to_show, globalOffset);
          me.x = data.xs;
          me.y = data.ys;
          fnParam = { ...fnParam, ...data };
        }
        await this.evalNode({
          parent: me,
          path: `${path}.${childKey}`,
          key: childKey,
          value: childValue,
          fnParam,
        });
      }
    } else if (isObjectOrArray(value)) {
      const me = (parent[key] = Array.isArray(value) ? [] : {});
      for (const [childKey, childValue] of Object.entries(value)) {
        await this.evalNode({
          parent: me,
          path: `${path}.${childKey}`,
          key: childKey,
          value: childValue,
          fnParam,
        });
      }
    } else {
      parent[key] = value;
    }
  }
  async update(input: {
    raw_config: any;
    visible_range: [number, number];
    hass: {};
    cssVars: {
      "--card-background-color": string;
      "--primary-background-color": string;
      "--primary-color": string;
      "--primary-text-color": string;
      "--secondary-text-color": string;
    };
  }) {
    let config = JSON.parse(JSON.stringify(input.raw_config));

    config = merge(config, defaultYaml, config);

    const themedLayout = getThemedLayout(
      input.cssVars,
      config.no_theme,
      config.no_default_layout
    );

    config.layout = merge(config.layout, themedLayout, config.layout);
    for (let i = 1; i < 31; i++) {
      const yaxis = "yaxis" + (i == 1 ? "" : i);
      merge(config.layout[yaxis], config.defaults?.yaxes, config.layout[yaxis]);
    }
    config.entities = config.entities.map((entity) => {
      if (typeof entity === "string") entity = { entity };
      const [oldAPI_entity, oldAPI_attribute] = entity.entity.split("::");
      if (oldAPI_attribute) {
        entity.entity = oldAPI_entity;
        entity.attribute = oldAPI_attribute;
      }
      merge(entity, defaultEntity, config.defaults?.entity, entity);
      return entity;
    });

    const fnParam = {
      vars: {},
      hass: input.hass,
      visible_range: input.visible_range,
    };
    this.busy = true;
    this.partiallyParsedConfig = {};
    for (const [key, value] of Object.entries(config)) {
      await this.evalNode({
        parent: this.partiallyParsedConfig,
        path: key,
        key: key,
        value,
        fnParam,
      });
    }
    this.busy = false;
  }
}
const parser = new ConfigParser();
const visible_range: [number, number] = [2, 4];
const hass = {};
const cssVars = {
  "--card-background-color": "--card-background-color",
  "--primary-background-color": "--primary-background-color",
  "--primary-color": "--primary-color",
  "--primary-text-color": "--primary-text-color",
  "--secondary-text-color": "--secondary-text-color",
};
await parser.update({
  raw_config: the_config,
  visible_range,
  hass,
  cssVars,
});
console.log(JSON.stringify(parser.config, null, 2));


{
  "refresh_interval": "auto",
  "offset": "0s",
  "hours_to_show": 1,
  "entities": [
    {
      "entity": "sensor.temperature",
      "attribute": "temperature",
      "x": [
        1,
        2,
        3,
        4
      ],
      "y": [
        21,
        22,
        23,
        24
      ],
      "offset": "0s",
      "unit_of_measurement": "W",
      "text": [
        "y=${y}",
        "y=${y}",
        "y=${y}",
        "y=${y}"
      ],
      "filters": [
        {
          "add": 5
        }
      ],
      "hovertemplate": "<b>%{customdata.name}</b><br><i>%{x}</i><br>%{y}%{customdata.unit_of_measurement}<extra></extra>",
      "mode": "lines",
      "show_value": true,
      "line": {
        "width": 1,
        "shape": "hv"
      },
      "internal": [
        1,
        0,
        1,
        0
      ]
    }
  ],
  "layout": {
    "title": {
      "y": 1,
      "pad": {
        "t": 15
      }
    },
    "legend": {
      "traceorder": "normal",
      "orientation": "h",

[graph](https://dreampuf.github.io/GraphvizOnline/#digraph%20G%20%7B%0A%20%20%20%20rankdir%3DTD%3B%0A%0A%20%20%20%20node%20%5Bfillcolor%3DMediumTurquoise%5D%0A%20%20%20%20plot%0A%20%20%20%20%0A%20%20%20%20node%20%5Bfontname%3D%22Courier%22%2Cfontsize%3D10.0%2Cshape%3Dcylinder%2Cstyle%3Dfilled%2Cfillcolor%3Dcadetblue%2Cfontcolor%3Dblack%2Cfontsize%3D15.0%5D%0A%20%20%20%20%23%20plotlyjs%20input%0A%20%20%20%20parsed_config%0A%20%20%20%20parsed_data%0A%20%20%20%20parsed_layout%0A%20%20%20%20%22plotlyjs%20input%22%0A%20%20%20%20%0A%20%20%20%20node%20%5Bshape%3Dcomponent%2Cstyle%3Dfilled%2Cfillcolor%3Dlightsalmon%2Cfontcolor%3Dblack%5D%0A%20%20%20%20%23%20card%20defaults%0A%20%20%20%20preset_config%0A%20%20%20%20preset_layout%0A%20%20%20%20themed_layout%0A%20%20%20%20entity_presets%0A%20%20%20%20%22card%20defaults%22%0A%20%20%20%20%0A%20%20%20%20node%20%5Bshape%3Dsignature%2Cfillcolor%3DPlum%2Cfontcolor%3Dblack%2Ccolor%3Dblack%5D%0A%20%20%20%20%23user%20confis%0A%20%20%20%20hours_to_show%0A%20%20%20%20axes_defaults%0A%20%20%20%20entity_defaults%0A%20%20%20%20global_offset%0A%20%20%20%20offset%0A%20%20%20%20entity_id%0A%20%20%20%20attribute%0A%20%20%20%20statistic%0A%20%20%20%20period%0A%20%20%20%20filters%0A%20%20%20%20%22user%20input%22%0A%20%20%20%20%0A%20%20%20%20node%20%5Bshape%3Dcomponent%2Cfillcolor%3Dlightblue%5D%0A%20%20%20%20%23internal%20nodes%0A%20%20%20%20visible_range%0A%20%20%20%20fetched_data%0A%20%20%20%20auto_period%0A%20%20%20%20auto_fetch_range%0A%20%20%20%20show_value_traces%0A%20%20%20%20units%0A%20%20%20%20axes%0A%20%20%20%20%22internal%22%0A%20%20%20%20node%20%5Bfillcolor%3Dred%5D%0A%20%20%20%20%0A%20%20%20%20subgraph%20clusterNotation%20%7B%0A%20%20%20%20%20%20%20label%3D%22Notation%22%3B%0A%20%20%20%20%20%20%20style%3Dfilled%3B%0A%20%20%20%20%20%20%20fillcolor%3Dlightgrey%0A%20%20%20%20%20%20%20%22internal%22%0A%20%20%20%20%20%20%20%22user%20input%22%0A%20%20%20%20%20%20%20%22card%20defaults%22%0A%20%20%20%20%20%20%20%22plotlyjs%20input%22%0A%20%20%20%20%7D%0A%20%20%20%20preset_config%20-%3E%20parsed_config%0A%0A%20%20%20%20preset_layout%20-%3E%20parsed_layout%0A%20%20%20%20themed_layout%20-%3E%20parsed_layout%0A%20%20%20%20axes_defaults%20-%3E%20axes%0A%0A%20%20%20%20global_offset%20-%3E%20fetched_data%0A%20%20%20%20offset%20-%3E%20fetched_data%0A%20%20%20%20entity_id%20-%3E%20fetched_data%0A%20%20%20%20attribute%20-%3E%20fetched_data%0A%20%20%20%20statistic%20-%3E%20fetched_data%0A%20%20%20%20period%20-%3E%20fetched_data%0A%20%20%20%20auto_period%20-%3E%20period%0A%20%20%20%20visible_range%20-%3E%20fetched_data%0A%20%20%20%20entity_defaults%20-%3E%20fetched_data%0A%20%20%20%20entity_presets%20-%3E%20fetched_data%20%0A%20%20%20%20fetched_data%20-%3E%20filters%0A%20%20%20%20%0A%20%20%20%20filters%20-%3E%20show_value_traces%0A%20%20%20%20show_value_traces%20-%3E%20parsed_data%0A%20%20%20%20filters%20-%3E%20parsed_data%0A%20%20%20%20entity_defaults%20-%3E%20show_value_traces%0A%20%20%20%20%0A%20%20%20%20hours_to_show%20-%3E%20auto_fetch_range%0A%20%20%20%20auto_fetch_range%20-%3E%20visible_range%0A%20%20%20%20global_offset%20-%3E%20auto_fetch_range%0A%20%20%20%20visible_range%20-%3E%20auto_period%0A%0A%20%20%20%20parsed_data%20-%3E%20units%0A%20%20%20%20units%20-%3E%20axes%0A%20%20%20%20axes%20-%3E%20parsed_layout%0A%20%20%20%20%0A%20%20%20%20%0A%0A%20%20%20%20parsed_config%20-%3E%20plot%0A%20%20%20%20parsed_data%20-%3E%20plot%0A%20%20%20%20parsed_layout%20-%3E%20plot%0A%7D)